In [1]:
import torch
import torch.tensor
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import torch.utils.data as Data
torch.manual_seed(1)

In [2]:
VOCAB_SIZE = 4346
EMBEDDING_SIZE = 200
HIDDEN_SIZE = 200
TARGET_SIZE = 2
SENTENCE_LEN = 103
BATCH_SIZE = 64

In [3]:
def get_myembed():
    myembed = []
    with open("new_vector.txt","r") as f:
        for line in f:
            words = line.split()
            for word in words:
                myembed.append(float(word))
    for i in range(EMBEDDING_SIZE):myembed.append(0)
    myembed = np.array(myembed)
    myembed = myembed.reshape((VOCAB_SIZE,EMBEDDING_SIZE))
    myembed = torch.from_numpy(myembed)
    return myembed

In [4]:
class SimpleLSTM(nn.Module):
    def __init__(self,vocab_size,embedding_size,hidden_size,target_size):
        super(SimpleLSTM,self).__init__()
        self.embed = nn.Embedding(vocab_size,embedding_size,padding_idx = 4345)
        self.lstm = nn.LSTM(embedding_size,hidden_size,batch_first = True)
        self.linear = nn.Linear(hidden_size,target_size)
        self.dropout = nn.Dropout(p=0.5)
        self.init_embed()
    def init_embed(self):
        self.embed.weight.data.copy_(get_myembed())
    def forward(self,sentence):
        myembed = self.embed(sentence)
        output,(hidden,c) = self.lstm(myembed.view(-1,SENTENCE_LEN,EMBEDDING_SIZE))
        
        predict = self.linear((output[:,-1,:].view(-1,HIDDEN_SIZE)))
        return predict        

In [5]:
def load_data():
    origin_y = []
    with open("input_label.txt","r") as f:
        for line in f:
            origin_y.append(int(line))
    origin_x = []
    with open("input_pad.txt","r") as f:
        for line in f:
            words = line.split()
            assert len(words) == SENTENCE_LEN,"sentence pad"
            for word in words:
                origin_x.append(int(word))
    numpx = np.array(origin_x)
    numpx = numpx.reshape(-1,SENTENCE_LEN)
    x = torch.from_numpy(numpx)   
    numpy = np.array(origin_y)
    y = torch.from_numpy(numpy)
    return x,y

In [6]:
def evaluate(predict,y):
    global tp 
    global fp 
    global fn
    tp += torch.masked_select(mask = y.byte(),input = predict).sum().cpu()
    fp += torch.masked_select(mask =~y.byte(),input = predict).sum().cpu()
    fn += torch.masked_select(mask =y.byte(),input = ~predict.byte()).sum().cpu()

In [7]:
tp = 0
fp = 0
fn = 0
predict = np.array([0,1,0,0,1,1,1])
y = np.array      ([1,0,1,1,0,0,0])
predict = torch.from_numpy(predict)
y = torch.from_numpy(y)
evaluate(predict,y)


In [8]:
def whole_eval():
    precision = tp.float() / (tp.float() + fp.float())
    recall = tp.float() / (tp.float() + fn.float())
    print(precision)
    print(recall)
    f1 = 2 * precision * recall / (precision + recall)
    return f1
            

In [45]:

x,y = load_data()


In [10]:

from collections import Counter

import numpy as np


from sklearn.datasets import load_iris

from imblearn.datasets import make_imbalance

from imblearn.over_sampling import RandomOverSampler
from imblearn.under_sampling import RandomUnderSampler
from imblearn.under_sampling import TomekLinks

In [11]:
sampling_strategy = {0: 4021, 1: 4021}
bx,by = make_imbalance(x, y,sampling_strategy=sampling_strategy)
print(bx,by)
bx = torch.from_numpy(bx)
by = torch.from_numpy(by)

[[1502 3955 3396 ... 4345 4345 4345]
 [2918 1101 3966 ... 4345 4345 4345]
 [3994 3075 4345 ... 4345 4345 4345]
 ...
 [4345 4345 1548 ... 4345 4345 4345]
 [1019 2875 1501 ... 4345 4345 4345]
 [1502 3955 3465 ... 4345 4345 4345]] [0 0 0 ... 1 1 1]


In [12]:

sampling_strategy = 'not majority'

ros = RandomOverSampler(sampling_strategy=sampling_strategy)
X_res, y_res = ros.fit_resample(x, y)
print(X_res.shape)
print(y_res.shape)
X_res = torch.from_numpy(X_res)
y_res = torch.from_numpy(y_res)

(47542, 103)
(47542,)


In [46]:
print(y_res.numpy().sum())
print(len(x))
torch_dataset = Data.TensorDataset(x,y)
#torch_dataset = Data.TensorDataset(x,y)
loader = Data.DataLoader(
    dataset = torch_dataset,
    batch_size = BATCH_SIZE,
    shuffle = True,
    num_workers = 2
)

23771
27792


In [47]:

mynet = SimpleLSTM(VOCAB_SIZE,EMBEDDING_SIZE,HIDDEN_SIZE,TARGET_SIZE)
mynet.cuda()
loss_function = nn.CrossEntropyLoss()
optimizer = optim.Adam(mynet.parameters(), lr=0.01)




In [15]:
print(by)

tensor([ 0,  0,  0,  ...,  1,  1,  1])


In [59]:

his_loss = []
wrongnum = 0
totalone = 0
for epoch in range(10):
    tp = 0
    fp = 0
    fn = 0
    wrongnum = 0
    totalone = 0
    whole_loss = 0.0
    print(len(loader))
    for step,(batch_x,batch_y)in enumerate(loader):
        #if(step % 100) == 0:print(step)
        #print(batch_x.shape)
        out = mynet(batch_x.cuda())
        pred = torch.max(F.softmax(out,dim = 1),1)[1]
        wrongnum += torch.sum(pred != batch_y.cuda())
        totalone += torch.sum(pred)
        loss = loss_function(out,batch_y.cuda())
        evaluate(pred.cuda(),batch_y.cuda())
        whole_loss += loss
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    print(wrongnum)
    print(totalone)
    print(whole_loss)
    his_loss.append(whole_loss)
    print("---------------")
    print(tp)
    print(fp)
    print(fn)
    print("!!!!!!!!!!!!!!!!!!!")
    #print(pred)
    #print(loss)

435
tensor(2125, device='cuda:0')
tensor(3424, device='cuda:0')
tensor(81.9255, device='cuda:0')
---------------
tensor(2660)
tensor(764)
tensor(1361)
!!!!!!!!!!!!!!!!!!!
435
tensor(2211, device='cuda:0')
tensor(3408, device='cuda:0')
tensor(83.3869, device='cuda:0')
---------------
tensor(2609)
tensor(799)
tensor(1412)
!!!!!!!!!!!!!!!!!!!
435
tensor(2162, device='cuda:0')
tensor(3397, device='cuda:0')
tensor(80.2593, device='cuda:0')
---------------
tensor(2628)
tensor(769)
tensor(1393)
!!!!!!!!!!!!!!!!!!!
435
tensor(2149, device='cuda:0')
tensor(3428, device='cuda:0')
tensor(80.2047, device='cuda:0')
---------------
tensor(2650)
tensor(778)
tensor(1371)
!!!!!!!!!!!!!!!!!!!
435
tensor(2132, device='cuda:0')
tensor(3365, device='cuda:0')
tensor(81.3907, device='cuda:0')
---------------
tensor(2627)
tensor(738)
tensor(1394)
!!!!!!!!!!!!!!!!!!!
435
tensor(2179, device='cuda:0')
tensor(3404, device='cuda:0')
tensor(82.0788, device='cuda:0')
---------------
tensor(2623)
tensor(781)
tensor(

In [40]:
tp = 0
fp = 0
fn = 0
wrongnum = 0
totalone = 0
for step,(batch_x,batch_y)in enumerate(loader):
        if(step % 100) == 0:print(step)
        out = mynet(batch_x.cuda())
        pred = torch.max(F.softmax(out,dim = 1),1)[1]
        #print(pred)
        #print(batch_y)
        evaluate(pred.cuda(),batch_y.cuda())
        wrongnum += torch.sum(pred != batch_y.cuda())
        totalone += torch.sum(pred)

0
100
200
300
400


In [58]:
print(his_loss)
print(wrongnum)
print(totalone)

[tensor(90.1833, device='cuda:0'), tensor(90.5938, device='cuda:0'), tensor(88.6918, device='cuda:0'), tensor(87.2920, device='cuda:0'), tensor(86.2437, device='cuda:0'), tensor(86.8085, device='cuda:0'), tensor(83.9423, device='cuda:0'), tensor(83.7342, device='cuda:0'), tensor(84.8563, device='cuda:0'), tensor(84.1593, device='cuda:0')]
tensor(2248, device='cuda:0')
tensor(3363, device='cuda:0')


In [56]:
print(tp)
print(fp)
print(fn)

tensor(2568)
tensor(795)
tensor(1453)


In [60]:
whole_eval()

tensor(0.7794)
tensor(0.6546)


tensor(0.7115)